In [2]:
import os
import pandas as pd
import numpy as np

from math import ceil 
from skimage import io
from skimage import filters, color
from scipy import ndimage as ndi
from PIL import Image
import cv2
import matplotlib
matplotlib.rcParams.update({'figure.max_open_warning': 0})
import matplotlib.pyplot as plt
import my_app.processing as proc

import tensorflow.keras as keras
# from tensorflow.keras import layers
# keras = tf.keras
# AUTOTUNE = tf.data.experimental.AUTOTUNE ## tf.data transformation parameters
# import matplotlib.pyplot as plt

In [5]:
N = -1
!ls saved_models
models = os.popen("ls saved_models").read().split('\n')[:-1]
tf_model = keras.models.load_model('saved_models/'+models[N])
MODEL_NAME = models[N][:-3]
MODEL_NAME

baseline_DecisionTreeClassifier_201911101573380701.h5
baseline_DecisionTreeClassifier_201911101573380991.p
mnist_5epochs_20191028.h5
mnist_hasyv2_20epochs_201910291572394063.h5
mnist_hasyv2_master_20epochs_201911081573208118.h5
mnist_hasyv2_master_20epochs_batch64_201911081573209782.h5
mnist_hasyv2_master_20epochs_batch64__ALLDATA_201911081573211546.h5


'mnist_hasyv2_master_20epochs_batch64__ALLDATA_201911081573211546'

In [3]:
folder = 'Kensanata/'
imgs = os.popen("ls "+folder).read().split('\n')[:-1]
for i,img in enumerate(imgs):
    imgs[i]=folder+img
imgs #= imgs[::-1]

['Kensanata/0001_CH4M0____0.png',
 'Kensanata/0001_CH4M0____1.png',
 'Kensanata/0001_CH4M0____2.png',
 'Kensanata/0001_CH4M0____3.png',
 'Kensanata/0001_CH4M0____4.png',
 'Kensanata/0001_CH4M0____5.png',
 'Kensanata/0001_CH4M0____6.png',
 'Kensanata/0001_CH4M0____7.png',
 'Kensanata/0001_CH4M0____8.png',
 'Kensanata/0001_CH4M0____9.png',
 'Kensanata/0001_CH4M10____0.png',
 'Kensanata/0001_CH4M10____1.png',
 'Kensanata/0001_CH4M10____2.png',
 'Kensanata/0001_CH4M10____3.png',
 'Kensanata/0001_CH4M10____4.png',
 'Kensanata/0001_CH4M10____5.png',
 'Kensanata/0001_CH4M10____6.png',
 'Kensanata/0001_CH4M10____7.png',
 'Kensanata/0001_CH4M10____8.png',
 'Kensanata/0001_CH4M10____9.png',
 'Kensanata/0001_CH4M1____0.png',
 'Kensanata/0001_CH4M11____0.png',
 'Kensanata/0001_CH4M11____1.png',
 'Kensanata/0001_CH4M11____2.png',
 'Kensanata/0001_CH4M11____3.png',
 'Kensanata/0001_CH4M11____4.png',
 'Kensanata/0001_CH4M11____5.png',
 'Kensanata/0001_CH4M11____6.png',
 'Kensanata/0001_CH4M11____7.pn

In [7]:
FILE.split('_')

['Kensanata/0001', 'CH4M0', '', '', '', '0.png']

In [8]:
country = []
age = []
gender = []
# def get_demographics(df):
for FILE in imgs:
    country.append(FILE.split('_')[1][:2])
    try:
        age.append(int(FILE.split('_')[1][2])*10)
    except:
        age.append(np.nan)
    gender.append(FILE.split('_')[1][3])

In [8]:
labels = []
for filename in imgs:
    for digit in filename.split('/')[1][-5:-4]:
        labels.append(int(digit))
labels[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
df = pd.DataFrame(imgs)
df.columns = ['filename']
df['label'] = labels
df['country'] = country
df['gender'] = gender
df['age'] = age
df_subset = df.copy()    #.iloc[:1000]
df_subset.head()

,filename,label,country,gender,age
0,unseen_set/0001_CH4M0____0.png,0,CH,M,40.0
1,unseen_set/0001_CH4M0____1.png,1,CH,M,40.0
2,unseen_set/0001_CH4M0____2.png,2,CH,M,40.0
3,unseen_set/0001_CH4M0____3.png,3,CH,M,40.0
4,unseen_set/0001_CH4M0____4.png,4,CH,M,40.0


In [10]:
SAMPLE = df_subset.sample(frac=0.5, replace=False,random_state=1)
SAMPLE

,filename,label,country,gender,age
11780,unseen_set/0021_CH4M23____6.png,6,CH,M,40.0
13154,unseen_set/0023_IT3M39____6.png,6,IT,M,30.0
3323,unseen_set/0007_CH3M13____0.png,0,CH,M,30.0
2355,unseen_set/0004_CH4M59____9.png,9,CH,M,40.0
8794,unseen_set/0016_CH3F22____4.png,4,CH,F,30.0
16371,unseen_set/0028_PL3F7____7.png,7,PL,F,30.0
11830,unseen_set/0021_CH4M28____1.png,1,CH,M,40.0
854,unseen_set/0002_CH5M32____2.png,2,CH,M,50.0
16864,unseen_set/0029_PL3M51____9.png,9,PL,M,30.0
6609,unseen_set/0012_CH3M39____3.png,3,CH,M,30.0


In [11]:
SAMPLE.country.value_counts()

CH    6088
PL     873
IT     616
AT     325
US     309
RO     286
Name: country, dtype: int64

In [12]:
predictions = []
newimg = []
with open(MODEL_NAME+'___unseenset__SAMPLE.txt','w') as writeFile:
    for index,image in enumerate(SAMPLE['filename']):
        matname = 'finaltests/SAMPLE___unseen_set'+MODEL_NAME+image.split('/')[1][:-4]
        print(index+1,'of',len(SAMPLE))
        print('processing',image)
        try:
            binary_arr,label_arr, segments,orig = proc.label_segments(image,matname,photo=False,marker=True)
            found = label_arr==1
            i=2
            a=[]
            maxx = np.sum(found.flatten())
            if len(segments)>1:
                while i<=len(segments):
                    if np.sum((label_arr==i).flatten())/i>maxx:
                        maxx = np.sum((label_arr==i).flatten())/i
                        found = label_arr==i
                    i+=1
            x,y = np.where(found)
            xmin,xmax,ymin,ymax = np.min(x),np.max(x),np.min(y),np.max(y)
            xlen,ylen = found[xmin:xmax,ymin:ymax].shape

            diff = np.abs(ylen-xlen)
            change = ceil(diff/2)
            if diff!=0:
                if ylen>xlen:
                    xmin-=change
                    xmax+=change

                else:
                    ymin-=change
                    ymax+=change

                xlen,ylen = xmax-xmin,ymax-ymin
                diff=np.abs(ylen-xlen)
                if xlen>ylen: ymax+=diff
                elif ylen>xlen: xmax+=diff
            digit = 1-binary_arr[xmin:xmax,ymin:ymax]
            digit = np.pad(digit,int(len(digit)*.2),mode= 'constant', constant_values=(0,0))   
            im = Image.fromarray(np.array(digit)*255.0).convert("RGB")
            im.save('000.jpg')
            img = cv2.resize(cv2.imread('000.jpg',cv2.IMREAD_GRAYSCALE),(28,28),interpolation=cv2.INTER_CUBIC)
            !rm 000.jpg
            p = np.argmax(tf_model.predict(img.astype(float).flatten().reshape((1, 28, 28, 1))))
            temp = matname+'___predicted____'+str(p)+'.jpg'
            im.save(temp)
            newimg.append(temp)
            predictions.append(p)
            plt.close('all')
        except:
            p=np.nan
            predictions.append(p)
        row = str(df_subset.iloc[index]['label'])+'; '+str(p)+'\n'
#         print(row)
        writeFile.write(row)

writeFile.close()


1 of 8497
processing unseen_set/0021_CH4M23____6.png
2 of 8497
processing unseen_set/0023_IT3M39____6.png
3 of 8497
processing unseen_set/0007_CH3M13____0.png
4 of 8497
processing unseen_set/0004_CH4M59____9.png
5 of 8497
processing unseen_set/0016_CH3F22____4.png
6 of 8497
processing unseen_set/0028_PL3F7____7.png
7 of 8497
processing unseen_set/0021_CH4M28____1.png
8 of 8497
processing unseen_set/0002_CH5M32____2.png
9 of 8497
processing unseen_set/0029_PL3M51____9.png
10 of 8497
processing unseen_set/0012_CH3M39____3.png
11 of 8497
processing unseen_set/0023_IT3M44____6.png
12 of 8497
processing unseen_set/0028_PL3F34____5.png
13 of 8497
processing unseen_set/0019_US5M23____7.png
14 of 8497
processing unseen_set/0016_CH3F33____9.png
15 of 8497
processing unseen_set/0007_CH3M13____7.png
16 of 8497
processing unseen_set/0009_CH4M38____2.png
17 of 8497
processing unseen_set/0015_CH2M41____6.png
18 of 8497
processing unseen_set/0025_RO4F33____1.png
19 of 8497
processing unseen_set/0016_

153 of 8497
processing unseen_set/0019_US5M0____9.png
154 of 8497
processing unseen_set/0010_CH3M16____2.png
155 of 8497
processing unseen_set/0016_CH3F17____7.png
156 of 8497
processing unseen_set/0009_CH4M18____1.png
157 of 8497
processing unseen_set/0028_PL3F10____1.png
158 of 8497
processing unseen_set/0006_CH4M9____2.png
159 of 8497
processing unseen_set/0008_CH3F19____3.png
160 of 8497
processing unseen_set/0015_CH2M13____3.png
161 of 8497
processing unseen_set/0004_CH4M28____4.png
162 of 8497
processing unseen_set/0023_IT3M26____1.png
163 of 8497
processing unseen_set/0009_CH4M27____0.png
164 of 8497
processing unseen_set/0020_CH4M50____2.png
165 of 8497
processing unseen_set/0015_CH2M57____6.png
166 of 8497
processing unseen_set/0015_CH2M8____6.png
167 of 8497
processing unseen_set/0027_PL3M13____5.png
168 of 8497
processing unseen_set/0005_CH3M27____7.png
169 of 8497
processing unseen_set/0007_CH3M51____4.png
170 of 8497
processing unseen_set/0014_CH3F36____2.png
171 of 8497
p

303 of 8497
processing unseen_set/0004_CH4M47____2.png
304 of 8497
processing unseen_set/0009_CH4M40____5.png
305 of 8497
processing unseen_set/0025_RO4F8____2.png
306 of 8497
processing unseen_set/0008_CH3F26____4.png
307 of 8497
processing unseen_set/0009_CH4M44____3.png
308 of 8497
processing unseen_set/0006_CH4M47____8.png
309 of 8497
processing unseen_set/0010_CH3M36____0.png
310 of 8497
processing unseen_set/0016_CH3F18____8.png
311 of 8497
processing unseen_set/0014_CH3F16____3.png
312 of 8497
processing unseen_set/0028_PL3F21____3.png
313 of 8497
processing unseen_set/0017_CH4M49____1.png
314 of 8497
processing unseen_set/0008_CH3F5____8.png
315 of 8497
processing unseen_set/0025_RO4F37____6.png
316 of 8497
processing unseen_set/0013_CH3M57____9.png
317 of 8497
processing unseen_set/0004_CH4M32____9.png
318 of 8497
processing unseen_set/0016_CH3F21____3.png
319 of 8497
processing unseen_set/0023_IT3M5____7.png
320 of 8497
processing unseen_set/0012_CH3M14____6.png
321 of 8497
p

453 of 8497
processing unseen_set/0015_CH2M20____4.png
454 of 8497
processing unseen_set/0006_CH4M39____9.png
455 of 8497
processing unseen_set/0023_IT3M21____6.png
456 of 8497
processing unseen_set/0008_CH3F40____5.png
457 of 8497
processing unseen_set/0016_CH3F24____3.png
458 of 8497
processing unseen_set/0006_CH4M42____0.png
459 of 8497
processing unseen_set/0012_CH3M36____6.png
460 of 8497
processing unseen_set/0009_CH4M34____9.png
461 of 8497
processing unseen_set/0022_AT3M41____0.png
462 of 8497
processing unseen_set/0028_PL3F53____5.png
463 of 8497
processing unseen_set/0006_CH4M59____2.png
464 of 8497
processing unseen_set/0024_IT3M47____7.png
465 of 8497
processing unseen_set/0025_RO4F28____9.png
466 of 8497
processing unseen_set/0011_CH2F17____5.png
467 of 8497
processing unseen_set/0026_CH5M39____6.png
468 of 8497
processing unseen_set/0021_CH4M2____7.png
469 of 8497
processing unseen_set/0014_CH3F26____1.png
470 of 8497
processing unseen_set/0029_PL3M54____0.png
471 of 8497

603 of 8497
processing unseen_set/0008_CH3F3____4.png
604 of 8497
processing unseen_set/0023_IT3M25____8.png
605 of 8497
processing unseen_set/0021_CH4M17____6.png
606 of 8497
processing unseen_set/0010_CH3M28____6.png
607 of 8497
processing unseen_set/0024_IT3M1____1.png
608 of 8497
processing unseen_set/0025_RO4F2____1.png
609 of 8497
processing unseen_set/0015_CH2M56____7.png
610 of 8497
processing unseen_set/0004_CH4M56____3.png
611 of 8497
processing unseen_set/0001_CH4M39____3.png
612 of 8497
processing unseen_set/0027_PL3M47____4.png
613 of 8497
processing unseen_set/0018_CHXX7____4.png
614 of 8497
processing unseen_set/0023_IT3M8____9.png
615 of 8497
processing unseen_set/0026_CH5M7____5.png
616 of 8497
processing unseen_set/0020_CH4M1____7.png
617 of 8497
processing unseen_set/0007_CH3M8____3.png
618 of 8497
processing unseen_set/0018_CHXX25____4.png
619 of 8497
processing unseen_set/0003_CH3M46____4.png
620 of 8497
processing unseen_set/0020_CH4M6____2.png
621 of 8497
process

...

7815 of 8497
processing unseen_set/0004_CH4M41____0.png
7816 of 8497
processing unseen_set/0026_CH5M30____7.png
7817 of 8497
processing unseen_set/0013_CH3M5____6.png
7818 of 8497
processing unseen_set/0004_CH4M27____7.png
7819 of 8497
processing unseen_set/0017_CH4M54____9.png
7820 of 8497
processing unseen_set/0020_CH4M8____4.png
7821 of 8497
processing unseen_set/0015_CH2M5____2.png
7822 of 8497
processing unseen_set/0001_CH4M18____4.png
7823 of 8497
processing unseen_set/0027_PL3M36____6.png
7824 of 8497
processing unseen_set/0003_CH3M50____2.png
7825 of 8497
processing unseen_set/0009_CH4M53____5.png
7826 of 8497
processing unseen_set/0009_CH4M11____2.png
7827 of 8497
processing unseen_set/0029_PL3M18____3.png
7828 of 8497
processing unseen_set/0022_AT3M39____5.png
7829 of 8497
processing unseen_set/0016_CH3F42____8.png
7830 of 8497
processing unseen_set/0018_CHXX2____3.png
7831 of 8497
processing unseen_set/0010_CH3M24____6.png
7832 of 8497
processing unseen_set/0026_CH5M33____6.

7963 of 8497
processing unseen_set/0022_AT3M22____6.png
7964 of 8497
processing unseen_set/0014_CH3F9____1.png
7965 of 8497
processing unseen_set/0019_US5M19____6.png
7966 of 8497
processing unseen_set/0018_CHXX3____1.png
7967 of 8497
processing unseen_set/0021_CH4M4____3.png
7968 of 8497
processing unseen_set/0020_CH4M42____5.png
7969 of 8497
processing unseen_set/0029_PL3M15____4.png
7970 of 8497
processing unseen_set/0020_CH4M53____0.png
7971 of 8497
processing unseen_set/0018_CHXX53____1.png
7972 of 8497
processing unseen_set/0001_CH4M47____4.png
7973 of 8497
processing unseen_set/0017_CH4M5____3.png
7974 of 8497
processing unseen_set/0002_CH5M1____7.png
7975 of 8497
processing unseen_set/0013_CH3M11____1.png
7976 of 8497
processing unseen_set/0024_IT3M14____3.png
7977 of 8497
processing unseen_set/0020_CH4M16____8.png
7978 of 8497
processing unseen_set/0019_US5M32____3.png
7979 of 8497
processing unseen_set/0029_PL3M23____2.png
7980 of 8497
processing unseen_set/0001_CH4M15____1.p

8110 of 8497
processing unseen_set/0009_CH4M33____7.png
8111 of 8497
processing unseen_set/0022_AT3M36____2.png
8112 of 8497
processing unseen_set/0001_CH4M13____7.png
8113 of 8497
processing unseen_set/0009_CH4M0____0.png
8114 of 8497
processing unseen_set/0029_PL3M20____0.png
8115 of 8497
processing unseen_set/0019_US5M11____7.png
8116 of 8497
processing unseen_set/0017_CH4M17____2.png
8117 of 8497
processing unseen_set/0003_CH3M11____0.png
8118 of 8497
processing unseen_set/0026_CH5M18____5.png
8119 of 8497
processing unseen_set/0026_CH5M34____9.png
8120 of 8497
processing unseen_set/0002_CH5M29____5.png
8121 of 8497
processing unseen_set/0026_CH5M42____4.png
8122 of 8497
processing unseen_set/0023_IT3M34____4.png
8123 of 8497
processing unseen_set/0001_CH4M4____8.png
8124 of 8497
processing unseen_set/0012_CH3M24____1.png
8125 of 8497
processing unseen_set/0002_CH5M28____2.png
8126 of 8497
processing unseen_set/0021_CH4M14____1.png
8127 of 8497
processing unseen_set/0022_AT3M17____

8264 of 8497
processing unseen_set/0010_CH3M1____5.png
8265 of 8497
processing unseen_set/0002_CH5M16____8.png
8266 of 8497
processing unseen_set/0020_CH4M17____7.png
8267 of 8497
processing unseen_set/0022_AT3M5____9.png
8268 of 8497
processing unseen_set/0015_CH2M17____6.png
8269 of 8497
processing unseen_set/0002_CH5M36____5.png
8270 of 8497
processing unseen_set/0026_CH5M5____0.png
8271 of 8497
processing unseen_set/0017_CH4M20____9.png
8272 of 8497
processing unseen_set/0006_CH4M48____4.png
8273 of 8497
processing unseen_set/0028_PL3F28____1.png
8274 of 8497
processing unseen_set/0005_CH3M16____3.png
8275 of 8497
processing unseen_set/0004_CH4M30____7.png
8276 of 8497
processing unseen_set/0024_IT3M21____8.png
8277 of 8497
processing unseen_set/0021_CH4M58____1.png
8278 of 8497
processing unseen_set/0024_IT3M49____9.png
8279 of 8497
processing unseen_set/0019_US5M17____8.png
8280 of 8497
processing unseen_set/0019_US5M53____1.png
8281 of 8497
processing unseen_set/0014_CH3F47____4

8414 of 8497
processing unseen_set/0027_PL3M16____7.png
8415 of 8497
processing unseen_set/0020_CH4M15____8.png
8416 of 8497
processing unseen_set/0023_IT3M21____9.png
8417 of 8497
processing unseen_set/0007_CH3M32____1.png
8418 of 8497
processing unseen_set/0016_CH3F8____2.png
8419 of 8497
processing unseen_set/0008_CH3F11____9.png
8420 of 8497
processing unseen_set/0006_CH4M15____8.png
8421 of 8497
processing unseen_set/0008_CH3F7____6.png
8422 of 8497
processing unseen_set/0029_PL3M14____0.png
8423 of 8497
processing unseen_set/0009_CH4M44____8.png
8424 of 8497
processing unseen_set/0022_AT3M59____0.png
8425 of 8497
processing unseen_set/0008_CH3F38____3.png
8426 of 8497
processing unseen_set/0019_US5M38____8.png
8427 of 8497
processing unseen_set/0011_CH2F14____1.png
8428 of 8497
processing unseen_set/0023_IT3M39____3.png
8429 of 8497
processing unseen_set/0006_CH4M54____4.png
8430 of 8497
processing unseen_set/0020_CH4M57____2.png
8431 of 8497
processing unseen_set/0015_CH2M4____9

In [14]:
SAMPLE['predict'] = predictions
SAMPLE

,filename,label,country,gender,age,predict
11780,unseen_set/0021_CH4M23____6.png,6,CH,M,40.0,6.0
13154,unseen_set/0023_IT3M39____6.png,6,IT,M,30.0,6.0
3323,unseen_set/0007_CH3M13____0.png,0,CH,M,30.0,0.0
2355,unseen_set/0004_CH4M59____9.png,9,CH,M,40.0,9.0
8794,unseen_set/0016_CH3F22____4.png,4,CH,F,30.0,9.0
16371,unseen_set/0028_PL3F7____7.png,7,PL,F,30.0,7.0
11830,unseen_set/0021_CH4M28____1.png,1,CH,M,40.0,4.0
854,unseen_set/0002_CH5M32____2.png,2,CH,M,50.0,2.0
16864,unseen_set/0029_PL3M51____9.png,9,PL,M,30.0,9.0
6609,unseen_set/0012_CH3M39____3.png,3,CH,M,30.0,3.0


In [35]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix, confusion_matrix
SAMPLE = SAMPLE[SAMPLE.predict.notnull()]
SAMPLE

,filename,label,country,gender,age,predict,compare
11780,unseen_set/0021_CH4M23____6.png,6,CH,M,40.0,6.0,1
13154,unseen_set/0023_IT3M39____6.png,6,IT,M,30.0,6.0,1
3323,unseen_set/0007_CH3M13____0.png,0,CH,M,30.0,0.0,1
2355,unseen_set/0004_CH4M59____9.png,9,CH,M,40.0,9.0,1
8794,unseen_set/0016_CH3F22____4.png,4,CH,F,30.0,9.0,0
16371,unseen_set/0028_PL3F7____7.png,7,PL,F,30.0,7.0,1
11830,unseen_set/0021_CH4M28____1.png,1,CH,M,40.0,4.0,0
854,unseen_set/0002_CH5M32____2.png,2,CH,M,50.0,2.0,1
16864,unseen_set/0029_PL3M51____9.png,9,PL,M,30.0,9.0,1
6609,unseen_set/0012_CH3M39____3.png,3,CH,M,30.0,3.0,1


In [34]:
print(classification_report(SAMPLE.label,SAMPLE.predict))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       805
           1       0.94      0.44      0.60       789
           2       0.95      0.91      0.93       825
           3       0.94      0.87      0.90       810
           4       0.75      0.87      0.80       827
           5       0.94      0.91      0.93       781
           6       0.95      0.90      0.92       827
           7       0.85      0.94      0.89       847
           8       0.87      0.90      0.89       849
           9       0.62      0.89      0.73       827

    accuracy                           0.86      8187
   macro avg       0.88      0.86      0.85      8187
weighted avg       0.88      0.86      0.85      8187



In [28]:
##8187 OUT OF 8497 PRODUCED A PREDICTION
print(classification_report(SAMPLE.label,SAMPLE.predict))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       805
           1       0.94      0.44      0.60       789
           2       0.95      0.91      0.93       825
           3       0.94      0.87      0.90       810
           4       0.75      0.87      0.80       827
           5       0.94      0.91      0.93       781
           6       0.95      0.90      0.92       827
           7       0.85      0.94      0.89       847
           8       0.87      0.90      0.89       849
           9       0.62      0.89      0.73       827

    accuracy                           0.86      8187
   macro avg       0.88      0.86      0.85      8187
weighted avg       0.88      0.86      0.85      8187



In [31]:
print(multilabel_confusion_matrix(SAMPLE.label,SAMPLE.predict))

[[[7348   34]
  [  53  752]]

 [[7376   22]
  [ 438  351]]

 [[7325   37]
  [  73  752]]

 [[7334   43]
  [ 107  703]]

 [[7118  242]
  [ 111  716]]

 [[7359   47]
  [  68  713]]

 [[7323   37]
  [  85  742]]

 [[7200  140]
  [  53  794]]

 [[7228  110]
  [  84  765]]

 [[6906  454]
  [  94  733]]]


In [38]:
confusion_df = pd.DataFrame(confusion_matrix(SAMPLE.label,SAMPLE.predict))
confusion_df

,0,1,2,3,4,5,6,7,8,9
0,752,2,3,0,0,2,14,0,8,24
1,5,351,10,1,230,1,3,52,62,74
2,1,1,752,2,0,1,1,33,3,31
3,3,0,4,703,0,5,0,25,4,66
4,9,9,4,1,716,2,14,2,2,68
5,0,0,2,11,6,713,0,10,4,35
6,2,2,0,0,1,6,742,1,10,63
7,0,3,5,3,1,7,0,794,0,34
8,6,0,3,5,1,6,2,2,765,59
9,8,5,6,20,3,17,3,15,17,733


In [29]:
SAMPLE['compare']= (SAMPLE.label==SAMPLE.predict)*1
# df_subset['compare_sets']= [set(x)==set(y) for x,y in list(zip(df_subset.label,df_subset.predict))]
SAMPLE.compare.value_counts()

1    7021
0    1166
Name: compare, dtype: int64

In [30]:
SAMPLE.describe()

,label,age,predict,compare
count,8187.000000,7878.000000,8187.00000,8187.000000
mean,4.544766,34.435136,4.97325,0.857579
std,2.874361,7.866445,2.88295,0.349503
min,0.000000,20.000000,0.00000,0.000000
25%,2.000000,30.000000,3.00000,1.000000
50%,5.000000,30.000000,5.00000,1.000000
75%,7.000000,40.000000,8.00000,1.000000
max,9.000000,50.000000,9.00000,1.000000


In [40]:
bynum = SAMPLE.groupby('label').agg({'compare':'mean'})
plt.bar(bynum.reset_index().label,bynum.reset_index().compare)
# SAMPLE.to_csv(MODEL_NAME+'_SAMPLE.csv',index=None)

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.

<BarContainer object of 10 artists>

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "/home/nina/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 834, in run
  File "</home/nina/anaconda3/lib/python3.7/site-packages/decorator.py:decorator-gen-23>", line 2, in writeout_cache
  File "/home/nina/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 58, in needs_sqlite
  File "/home/nina/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 780, in writeout_cache
  File "/home/nina/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 764, in _writeout_input_cache
sqlite3.OperationalError: unable to open database file

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nina/anaconda3/lib/python3.7/site-packages/ipykernel/iostream.py", line 97, in _event_pipe
AttributeError: '_thread._local' object has no attribute 'event_pipe'

During handling of the abov

ImportError: /home/nina/anaconda3/lib/python3.7/site-packages/matplotlib/_png.cpython-37m-x86_64-linux-gnu.so: cannot open shared object file: Too many open files

<Figure size 432x288 with 1 Axes>

In [ ]:
bygender = SAMPLE.groupby('gender').agg({'compare':'mean'})
bygender

In [ ]:
bycountry = SAMPLE.groupby('country').agg({'compare':'mean'})
plt.bar(bycountry.reset_index().country,bycountry.reset_index().compare)

In [ ]:
folderz = 'finaltests/master_20epochs_batch64_20191108157320978200/'
trainz = []
labelz = []
imgsz = os.popen("ls "+folderz).read().split('\n')[:-1]
for i,img in enumerate(imgsz):
    imgsz[i]=folderz+img
    hmm = img.split('____')
    if hmm[2][0]==hmm[1][0]: 
        trainz.append(folderz+img)
        labelz.append(int(hmm[1][0]))

folderz = 'finaltests/unseen_setmnist_5epochs_2019102800/'
imgsz = os.popen("ls "+folderz).read().split('\n')[:-1]
for i,img in enumerate(imgsz):
    hmm = img.split('____')
    if hmm[2][0]==hmm[1][0]: 
        trainz.append(folderz+img)
        labelz.append(int(hmm[1][0]))

In [ ]:
def make_pixel_array(df):
    im = Image.open(df['path'],'r').convert('L')
    return cv2.resize(cv2.imread(df['path'],cv2.IMREAD_GRAYSCALE),(28,28),interpolation=cv2.INTER_CUBIC)
#     return np.asarray(im)  ##255-val inverts colors

In [ ]:
addtotrain.iloc[0]['array'].shape

In [ ]:
addtotrain = pd.DataFrame(trainz)
addtotrain.columns = ['path']
addtotrain['array']=addtotrain.apply(make_pixel_array,axis=1)
addtotrain['label']=labelz
addtotrain

In [ ]:
addtotrain.to_csv('NewTrainLabeledData.csv',index=None)

In [ ]:
predictions = []
newimg = []
with open(MODEL_NAME+'___unseenset.txt','w') as writeFile:
    for index,image in enumerate(df_subset['filename']):
        matname = 'finaltests/unseen_set'+MODEL_NAME+image.split('/')[1][:-4]
        print(index+1,'of',len(df_subset), '[processing',image,']')
        binary_arr,label_arr, segments,orig = proc.label_segments(image,matname,photo=False,marker=False)
        found = label_arr==1
        i=2
        a=[]
        maxx = np.sum(found.flatten())
        if len(segments)>1:
            while i<=len(segments):
                if np.sum((label_arr==i).flatten())/i>maxx:
                    maxx = np.sum((label_arr==i).flatten())/i
                    found = label_arr==i
                i+=1
        x,y = np.where(found)
        xmin,xmax,ymin,ymax = np.min(x),np.max(x),np.min(y),np.max(y)
        xlen,ylen = found[xmin:xmax,ymin:ymax].shape

        diff = np.abs(ylen-xlen)
        change = ceil(diff/2)
        if diff!=0:
            if ylen>xlen:
                xmin-=change
                xmax+=change

            else:
                ymin-=change
                ymax+=change

            xlen,ylen = xmax-xmin,ymax-ymin
            diff=np.abs(ylen-xlen)
            if xlen>ylen: ymax+=diff
            elif ylen>xlen: xmax+=diff
        digit = 1-binary_arr[xmin:xmax,ymin:ymax]
        digit = np.pad(digit,int(len(digit)*.2),mode= 'constant', constant_values=(0,0))   
        im = Image.fromarray(np.array(digit)*255.0).convert("RGB")
        im.save('000.jpg')
        img = cv2.resize(cv2.imread('000.jpg',cv2.IMREAD_GRAYSCALE),(28,28),interpolation=cv2.INTER_CUBIC)
        !rm 000.jpg
        p = np.argmax(tf_model.predict(img.astype(float).flatten().reshape((1, 28, 28, 1))))
        temp = matname+'___predicted____'+str(p)+'.jpg'
        im.save(temp)
        newimg.append(temp)
        predictions.append(p)
        plt.close('all')

        row = str(df_subset.iloc[index]['label'])+'; '+str(p)+'\n'
        print(row)
        writeFile.write(row)

writeFile.close()


In [ ]:
len(predictions)

In [ ]:
df_subset['predict'] = predictions
df_subset

In [ ]:
df_subset['compare']= (df_subset.label==df_subset.predict)*1
# df_subset['compare_sets']= [set(x)==set(y) for x,y in list(zip(df_subset.label,df_subset.predict))]
df_subset.compare.value_counts()

In [ ]:
bynum = df_subset.groupby('label').agg({'compare':'mean'})
plt.bar(bynum.reset_index().label,bynum.reset_index().compare)

In [ ]:
bygender = df_subset.groupby('gender').agg({'compare':'mean'})
bygender

In [ ]:
plt.bar(bynum.reset_index().label,bynum.reset_index().compare)

In [ ]:
df_subset.describe()

In [ ]:
def score_prediction(a1,b1):
    a=a1.copy()
    b=b1.copy()
    mishape=False
    if len(a)!=len(b):
        mishape=True
    over = len(a)
    score = 0

    for i in range(len(a)):
#         print(a,b)
        if a[i]==b[i]:
            score+=1
            continue
        elif mishape:
            b.remove(b[i])
            if len(a)==len(b):
                mishape=False
            if a[i]==b[i]:
                score+=1
            else:
                pass

#     print(score,' of ',over)
#     print(a)
#     print(b)
    return score/over

In [ ]:
df_subset['score'] = [score_prediction(a,b) for a,b in list(zip(df_subset['label'],df_subset['predict']))]

In [ ]:
actual2 = []
predict2 = []

for i in range(len(df_subset)):
    actual2.append(np.array(df_subset)[i][1])
    predict2.append(list(df_subset.iloc[i]['predict'])    )


In [ ]:
results = []
for i in range(len(df_subset)):
    results.append((np.array(actual2)[i]==np.array(predict2)[i])*1)
np.sum(np.sum(results))/(len(df_subset)*4)

In [ ]:
count_correct = []
for arr in results:
    count_correct.append(np.sum(arr))


In [ ]:
all_correct = count_correct.count(4)
correct_3 = count_correct.count(3)
correct_2 = count_correct.count(2)
correct_1 = count_correct.count(1)
none_correct = count_correct.count(0)

In [ ]:
print(all_correct,correct_3,correct_2,correct_1,none_correct)

In [ ]:
N=len(results)/100

In [ ]:
print(all_correct/N,correct_3/N,correct_2/N,correct_1/N,none_correct/N)

In [ ]:
df_subset.describe()

In [ ]:
df_subset.to_csv(MODEL_NAME+'___pregen.csv',index=None)

In [ ]:
list(df_subset[df_subset.compare]['filename'])

In [ ]:
# a = labels[-1]
# b = predictions[-1]

In [ ]:
arr = io.imread('testmarker/97765113.jpg')
arr.shape

In [ ]:
n=len(segments)
fig,axes = plt.subplots(n,figsize=(6*n,20))
for i,seg in enumerate(segments):
    x_center, y_center, xlen, ylen = newproc.crop_image(seg, label_arr, binary_arr,ax=axes[i])
    print(x_center, y_center, xlen, ylen)
    # plt.imshow(arr)
    # plt.axhline(x_center*x_height,color='r')
    # plt.axvline(y_center*y_height,color='r');

In [ ]:
df_subset[df_subset.compare]

In [ ]:
predictions = []
with open('test_new_model__subset.txt','w') as writeFile:
    for index,image in enumerate(df_subset['filename'].iloc[1:]):
        matname = 'tf22/HANDWRITING__'+image[-8:-4]
        print(index+1,'of',len(df_subset))
        print('processing ',image)
        binary_arr,label_arr, segments,orig = proc.label_segments(image,matname,photo=True)
        predicted = []
        for seg in segments:
            try:
                pred = proc.crop_image(seg,label_arr,orig,model=tf_model,direc=matname,svc=False,tf=True)[1]
            except:
                pred = None
            if pred!= None: predicted.append(pred)
        predictions.append(predicted)
        row = str(df_subset.iloc[index]['label'])+'; '+str(list(pred))+'\n'
        print(row)
        writeFile.write(row)
writeFile.close()


In [ ]:
# df_subset.to_csv('test_pred_tf3.csv',index=False)
predictions

In [ ]:
image = io.imread(imgs[0])
alpha=1.2
beta=0
marker = False
image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
if marker==False:
    image = cv2.blur(image, (8, 8))
arrraaayyy = np.array(image)
fig,axes = plt.subplots(1,figsize=(20,10))
label_arr, num_seg = ndi.label(color.rgb2gray(arrraaayyy))
axes.imshow(label_arr)
axes.set_title('Preprocessed Binary Image') 


In [ ]:
def plot_numbered_image(label_arr,savename='',no_rotate=False):
    IMAGE = savename+'_segmented.jpg'
    pixarray=np.rot90(label_arr,3)
    imax,jmax = pixarray.shape
    fig,ax=plt.subplots(ncols=1, nrows=1, figsize=(20,int(20*jmax/imax)))
    plt.xticks(np.arange(0,imax))
    plt.yticks(np.arange(0,jmax))
    for i in range(0,imax,2):
        for j in range(0,jmax,2):
            val = pixarray[i][j]
            if val != 0:
                ax.scatter(i,j,s=300,color='k')
#                 ax.text(i,j,8,fontsize=20)
#     ax.patch.set_facecolor('black')
    ax.set_axis_off()
    plt.xticks([])
    plt.yticks([])            
    plt.show()
    fig.savefig(IMAGE)
    
#     ORIG = cv2.imread(IMAGE)    
#     kernel = np.ones((10,10),np.float32)/25
#     GAUSS = cv2.blur(ORIG, (25, 25))    
# #     GAUSS = cv2.filter2D(ORIG,-1,kernel)
# #     GAUSS=cv2.medianblur(IMAGE,(5,5))
#     io.imsave(IMAGE,GAUSS)    
    
#     BLACK = plt.imread(IMAGE)
#     INVERT = cv2.bitwise_not(BLACK)
#     io.imsave(savename+'_segmented.jpg',INVERT)
#     return INVERT

In [ ]:
from math import ceil 
import cv2
from PIL import Image
IMAGE = 'my_app/data/predictions__img_original_from_photo.png'          ###imgs[0]
plt.imshow(io.imread(IMAGE))
binary_arr,label_arr, segments,orig = proc.label_segments(IMAGE,'',photo=True)
plt.imshow(np.array(binary_arr, dtype = np.uint8 ))

predictions = []
fig,axes = plt.subplots(len(segments),figsize=(10,60))
for seg,ax in list(zip(segments,axes.flatten())):
    found = label_arr==seg
    x,y = np.where(found)
    xmin,xmax,ymin,ymax = np.min(x),np.max(x),np.min(y),np.max(y)
    xlen,ylen = found[xmin:xmax,ymin:ymax].shape
    diff = np.abs(ylen-xlen)
    change = ceil(diff/2)
    if diff!=0:
        if ylen>xlen:
            xmin-=change
            xmax+=change

        else:
            ymin-=change
            ymax+=change

        xlen,ylen = xmax-xmin,ymax-ymin
        diff=np.abs(ylen-xlen)
        if xlen>ylen: ymax+=diff
        elif ylen>xlen: xmax+=diff
    digit = binary_arr[xmin:xmax,ymin:ymax]
    digit = np.pad(digit,int(len(digit)*.2),mode= 'constant', constant_values=(0,0))        
    

    if digit.shape[0]<10:
        ax.set_visible(False)       
        pass
    else:
        ax.imshow(digit)
        im = Image.fromarray(np.array(digit)*255.0).convert("RGB")
        im.save('000.png')
        img = cv2.resize(cv2.imread('000.png',cv2.IMREAD_GRAYSCALE),(28,28),interpolation=cv2.INTER_CUBIC)
        p = np.argmax(tf_model.predict(img.astype(float).flatten().reshape((1, 28, 28, 1))))
        ax.set_title(p)
        predictions.append([ymin,p])

predictions.sort()    
predictions = [pr[1] for pr in predictions]
print(IMAGE)
print(predictions)

In [ ]:
ORIG = io.imread(IMAGE)
GAUSS = cv2.blur(ORIG, (10, 10)) 
io.imshow(GAUSS)

In [ ]:
results

In [ ]:
from skimage import io, filters, color
import cv2
BLACK = io.imread('_segmented.jpg')
INVERT = cv2.bitwise_not(BLACK)
io.imshow(INVERT)

In [ ]:
for img in 

In [ ]:
import tensorflow as tf